In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import (
    ResNet50,
    ResNet101,
    VGG16,
    VGG19,
    EfficientNetB0,
    EfficientNetB2,
    EfficientNetV2B0,
    EfficientNetV2B2,
    InceptionV3,
    InceptionResNetV2
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


# Function to build each model with the given architecture
def build_model(base_model_constructor, input_shape=(48,48,3), num_classes=7):
    base_model = base_model_constructor(weights='imagenet', include_top=False, input_shape=input_shape)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Dictionary of architectures to train
model_dict = {
    'ResNet50': ResNet50,
    'ResNet101': ResNet101,
    'VGG16': VGG16,
    'VGG19': VGG19,
    'EfficientNetB0': EfficientNetB0,
    'EfficientNetB2': EfficientNetB2,
    'EfficientNetV2B0': EfficientNetV2B0,
    'EfficientNetV2B2': EfficientNetV2B2,
    'InceptionV3': InceptionV3,
    'InceptionResNetV2': InceptionResNetV2
}

# Adjust these according to your dataset size
train_steps = int(28221 / 32)
val_steps   = int(5641 / 32)
epochs      = 5

results = []

checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-5,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

# Train each model and store final validation accuracy
for model_name, constructor in model_dict.items():
    print(f"\nTraining model: {model_name}")
    model = build_model(constructor)
    
    history = model.fit(
        train_generator,
        steps_per_epoch=train_steps,
        epochs=epochs,
        callbacks=[reduce_lr, checkpoint, early_stopping],
        validation_data=validation_generator,
        validation_steps=val_steps,
        verbose=1
    )
    
    final_val_acc = history.history['val_accuracy'][-1] * 100
    results.append((model_name, final_val_acc))

# Extract model names and accuracies for plotting
model_names = [res[0] for res in results]
accuracies = [res[1] for res in results]

# Plot the results
plt.figure(figsize=(10, 6))
bars = plt.bar(model_names, accuracies, color='skyblue')

# Annotate each bar with its accuracy value
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height + 1,
        f"{height:.2f}%",
        ha='center',
        va='bottom'
    )

plt.xlabel('Modele')
plt.ylabel('Dokładność (%)')
plt.title('Dokładność modeli CNN')
plt.ylim([0, 100])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
